In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd

In [4]:
df = pd.read_csv('/content/thai_romanization.csv',delimiter='	', nrows = 100000)

df.to_csv('thai.txt', header=None, index=None, sep=' ', mode='a')

In [5]:
with open('/content/thai.txt', 'r') as f:
    text = f.read()

In [6]:
text[:100]

'กองพันทหารปืนใหญ่ kongphanthahanpuenyai\nวิฑูรย์ withun\nเมตาบอลิสม metabolisom\nบ้านหนองเลา bannonglao'

In [7]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [8]:
encoded[:100]

array([ 84,  38,  87,  13, 101,  75,  83,  73,   5,  64,  46,  20,  75,
         8,  73,   0,  33,  16,  21,  31,  34,  88,  78,  41,  69,  34,
        93,  41,  69,  41,  69,  34,  78,  55,  29,  34,   9,  69,  25,
        12,  28, 102,  76,  35,  64,  32,  81,  16,  98,  25,  93,  41,
        55,  34,  12,  62,  60,  53,   5,  96,  38,  89, 102, 100,  60,
        16,  36,  29,  93,  69,  82,  31,  40,  25,  45,  31,  36,  12,
        96,  54,   5,  75,  73,  75,  38,  87,  62,  89,   5,  16,  82,
        69,  34,  34,  31,  34,  88,  40,  69,  31])

In [9]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [10]:
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [11]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


In [12]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [13]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 84  38  87  13 101  75  83  73   5  64]
 [ 75  73  75  38  87  75  85   8 100  16]
 [ 48  59  38  16  56  75  75   5  84  92]
 [  3  29  93  78  41  37  69  72  41  55]
 [ 93  45  69  12  96  54   5  75  50  60]
 [ 44  65  84  84  23  54  16  21  41  55]
 [ 93  21  41  69  12  96  54   5  75  56]
 [ 12  61  38  32  60 102  53  64  59  60]]

y
 [[ 38  87  13 101  75  83  73   5  64  46]
 [ 73  75  38  87  75  85   8 100  16  82]
 [ 59  38  16  56  75  75   5  84  92  48]
 [ 29  93  78  41  37  69  72  41  55  34]
 [ 45  69  12  96  54   5  75  50  60  16]
 [ 65  84  84  23  54  16  21  41  55  21]
 [ 21  41  69  12  96  54   5  75  56  75]
 [ 61  38  32  60 102  53  64  59  60  53]]


In [14]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')


Training on GPU!


In [90]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        # print(hidden)
        # h0 = torch.zeros(self.n_layers, x.size(0), self.n_hidden).cuda() # 2 for bidirection 
        # c0 = torch.zeros(self.n_layers, x.size(0), self.n_hidden).cuda()
        # print(h0.shape)
        # Forward propagate LSTM
        # out, _ = self.lstm(x, (h0, c0))
        r_output, hidden = self.lstm(x, hidden)
        # r_output, hidden = self.lstm(x, torch.cat((h0, c0)))
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        


In [91]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [92]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(104, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=104, bias=True)
)


In [93]:
batch_size = 128
seq_length = 100
n_epochs = 10 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)


Epoch: 1/10... Step: 10... Loss: 3.9956... Val Loss: 3.9424
Epoch: 1/10... Step: 20... Loss: 3.9036... Val Loss: 3.8873
Epoch: 1/10... Step: 30... Loss: 3.9059... Val Loss: 3.8805
Epoch: 1/10... Step: 40... Loss: 3.8914... Val Loss: 3.8784
Epoch: 1/10... Step: 50... Loss: 3.8979... Val Loss: 3.8685
Epoch: 1/10... Step: 60... Loss: 3.8365... Val Loss: 3.8058
Epoch: 1/10... Step: 70... Loss: 3.6324... Val Loss: 3.6892
Epoch: 1/10... Step: 80... Loss: 3.4367... Val Loss: 3.3892
Epoch: 1/10... Step: 90... Loss: 3.2803... Val Loss: 3.2130
Epoch: 1/10... Step: 100... Loss: 3.1405... Val Loss: 3.1090
Epoch: 1/10... Step: 110... Loss: 3.0654... Val Loss: 3.0518
Epoch: 1/10... Step: 120... Loss: 3.0011... Val Loss: 2.9714
Epoch: 1/10... Step: 130... Loss: 2.9374... Val Loss: 2.8946
Epoch: 1/10... Step: 140... Loss: 2.8555... Val Loss: 2.8001
Epoch: 1/10... Step: 150... Loss: 2.7765... Val Loss: 2.7259
Epoch: 2/10... Step: 160... Loss: 2.7278... Val Loss: 2.6714
Epoch: 2/10... Step: 170... Loss:

In [94]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [95]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h


In [96]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars).split('\n')[0]

In [97]:
samp = df.sample(10)
thai = list(samp.word.values)
rom = list(samp.romanization.values)

In [98]:
res = []
for th in thai:
  res.append(sample(net, 100, prime=th, top_k=5).split()[1])

In [99]:
res

['binsapkhai',
 'thairuatchamchraochai',
 'khaochaophon',
 'banphangpan',
 'thambotnaekklangkai',
 'phuanokhup-iankhongnang',
 'khomrikthon',
 'kanonkhophaikhongnonyitun',
 'ban-alao',
 'banphaitanphasa']

In [100]:
pd.DataFrame([thai,res,rom]).transpose()

,0,1,2
0,สับไพ่,binsapkhai,sapphai
1,ตรวจชำระ,thairuatchamchraochai,truatchamra
2,เจ้าพ่อ,khaochaophon,chaopho
3,บ้านพังเป็ด,banphangpan,banphangpet
4,ตำบลแก่งกระจาน,thambotnaekklangkai,tambonkaengkrachan
5,ผู้ออกเสียงลงคะแนน,phuanokhup-iankhongnang,phu-oksianglongkhanaen
6,คริปตอน,khomrikthon,khripton
7,นกพงใหญ่พันธุ์ญี่ปุ่น,kanonkhophaikhongnonyitun,nokphongyaiphanyipun
8,อาฬหก,ban-alao,alahok
9,บ้านไผ่ตาสุ่ม,banphaitanphasa,banphaitasum
